## Setup Models

In [1]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings


#Setup the LLM
model = ChatOpenAI(model="gpt-5")

#Setup the Embedding
embedding = OpenAIEmbeddings(model="text-embedding-3-small")


## 03.02. Add  Product Pricing function tool

In [2]:
import pandas as pd
from langchain_core.tools import tool

#Load the laptop product pricing CSV into a Pandas dataframe.
product_pricing_df = pd.read_csv("data/Laptop pricing.csv")
print(product_pricing_df)

@tool
def get_laptop_price(laptop_name:str) -> int :
    """
    This function returns the price of a laptop, given its name as input.
    It performs a substring match between the input name and the laptop name.
    If a match is found, it returns the price of the laptop.
    If there is NO match found, it returns -1
    """

    #Filter Dataframe for matching names
    match_records_df = product_pricing_df[
                        product_pricing_df["Name"].str.contains(
                                                "^" + laptop_name, case=False)
                        ]
    #Check if a record was found, if not return -1
    if len(match_records_df) == 0 :
        return -1
    else:
        return match_records_df["Price"].iloc[0]

#Test the tool. Before running the test, comment the @tool annotation
# print(get_laptop_price("alpha"))
# print(get_laptop_price("testing"))


            Name  Price  ShippingDays
0  AlphaBook Pro   1499             2
1     GammaAir X   1399             7
2  SpectraBook S   2499             7
3   OmegaPro G17   2199            14
4  NanoEdge Flex   1699             2


## 03.03. Add Product Features Retrieval Tool

In [3]:
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the product featuers document.
loader=PyPDFLoader("./data/Laptop product descriptions.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
splits = text_splitter.split_documents(docs)

#Create a vector store with Chroma
prod_feature_store = Chroma.from_documents(
    documents=splits,
    embedding=embedding
)

get_product_features = create_retriever_tool(
    prod_feature_store.as_retriever(search_kwargs={"k": 1}),
    name="Get_Product_Features",
    description="""
    This store contains details about Laptops. It lists the available laptops
    and their features including CPU, memory, storage, design and advantages
    """
)

#Test the product feature store
print(prod_feature_store.as_retriever().invoke("Tell me about the AlphaBook Pro") )

[Document(id='6cb58d59-bf0c-4242-b397-afd89a131e02', metadata={'page': 0, 'source': './data/Laptop product descriptions.pdf', 'creationdate': 'D:20241011143914', 'total_pages': 1, 'page_label': '1', 'creator': 'PyPDF', 'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/'}, page_content='Fictional Laptop Descriptions\nAlphaBook Pro\nThe AlphaBook Pro is a sleek ultrabook with a 12th Gen Intel i7 processor, 16GB of DDR4 RAM,\nand a fast 1TB SSD. Ideal for professionals on the go, this laptop offers an impressive blend of\npower and portability.\nGammaAir X\nGammaAir X combines an AMD Ryzen 7 processor with 32GB of DDR4 memory and a 512GB\nNVMe SSD. Its thin and light form factor makes it perfect for users who need high performance in a\nportable design.\nSpectraBook S\nDesigned for power users, SpectraBook S features an Intel Core i9 processor, 64GB RAM, and a\nmassive 2TB SSD. This workstation-class laptop is perfect for intensive tasks like video editing and\n3D rendering.\nOmegaPr

## 03.04.Setup a Product QnA chatbot

In [4]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create a System prompt to provide a persona to the chatbot
system_prompt = SystemMessage("""
    You are professional chatbot that answers questions about laptops sold by your company.
    To answer questions about laptops, you will ONLY use the available tools and NOT your own memory.
    You will handle small talk and greetings by producing professional responses.
    """
)

#Create a list of tools available
tools = [get_laptop_price, get_product_features]

#Create memory across questions in a conversation (conversation memory)
checkpointer=MemorySaver()

#Create a Product QnA Agent. This is actual a graph in langGraph
product_QnA_agent=create_react_agent(
                                model=model, #LLM to use
                                tools=tools, #List of tools to use
                                prompt=system_prompt, #The system prompt
                                debug=False, #Debugging turned on if needed
                                checkpointer=checkpointer #For conversation memory
)


In [5]:
#Setup chatbot
import uuid
#To maintain memory, each request should be in the context of a thread.
#Each user conversation will use a separate thread ID
config = {"configurable": {"thread_id": uuid.uuid4()}}

#Test the agent with an input
inputs = {"messages":[
                HumanMessage("What are the features and pricing for GammaAir?")
            ]}

#Use streaming to print responses as the agent  does the work.
#This is an alternate way to stream agent responses without waiting for the agent to finish
for stream in product_QnA_agent.stream(inputs, config, stream_mode="values"):
    message=stream["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What are the features and pricing for GammaAir?
================================== Ai Message ==================================
Tool Calls:
  Get_Product_Features (call_5YGwp0CdNKyiK61p5EsUoyVu)
 Call ID: call_5YGwp0CdNKyiK61p5EsUoyVu
  Args:
    query: GammaAir
  get_laptop_price (call_zR5t5LypajAr4N683uw0HirW)
 Call ID: call_zR5t5LypajAr4N683uw0HirW
  Args:
    laptop_name: GammaAir
================================= Tool Message =================================
Name: get_laptop_price

1399
================================== Ai Message ==================================

Here’s what I have for GammaAir:

- Model in catalog: GammaAir X
- CPU: AMD Ryzen 7
- Memory: 32GB DDR4
- Storage: 512GB NVMe SSD
- Design: Thin and light form factor; high performance in a portable design

Price: $1,399

If you were looking for a different GammaAir variant, let me know and I’ll check. Would you like to compare it with

## 03.05. Execute the Product QnA Chatbot

In [6]:
import uuid
#Send a sequence of messages to chatbot and get its response
#This simulates the conversation between the user and the Agentic chatbot
user_inputs = [
    "Hello",
    "I am looking to buy a laptop",
    "Give me a list of available laptop names",
    "Tell me about the features of  SpectraBook",
    "How much does it cost?",
    "Give me similar information about OmegaPro",
    "What info do you have on AcmeRight ?",
    "Thanks for the help"
]

#Create a new thread
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    #Format the user message
    user_message = {"messages":[HumanMessage(input)]}
    #Get response from the agent
    ai_response = product_QnA_agent.invoke(user_message,config=config)
    #Print the response
    print(f"AGENT : {ai_response['messages'][-1].content}")


----------------------------------------
USER : Hello
AGENT : Hello! How can I help you today? If you’re shopping for a laptop, I can recommend models, compare specs, and share current pricing.
----------------------------------------
USER : I am looking to buy a laptop
AGENT : Great! I can help you find the right laptop. A few quick questions to narrow it down:

- Budget range?
- Primary use (e.g., office/work, student, programming, content creation, gaming)?
- Preferred screen size and weight/portability needs?
- Battery life importance (e.g., all‑day vs. mostly plugged in)?
- Operating system preference (Windows, macOS, ChromeOS)?
- Any must‑have specs or features (RAM/storage amount, dedicated GPU, touchscreen/2‑in‑1, ports)?
- Do you need it urgently or is timing flexible?

Share whatever you know, and I’ll recommend specific models and pricing.
----------------------------------------
USER : Give me a list of available laptop names
AGENT : Here are the available laptop models:
- 

In [7]:
#conversation memory by user
def execute_prompt(user, config, prompt):
    inputs = {"messages":[("user",prompt)]}
    ai_response = product_QnA_agent.invoke(inputs,config=config)
    print(f"\n{user}: {ai_response['messages'][-1].content}")

#Create different session threads for 2 users
config_1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
config_2 = {"configurable": {"thread_id": str(uuid.uuid4())}}

#Test both threads
execute_prompt("USER 1", config_1, "Tell me about the features of  SpectraBook")
execute_prompt("USER 2", config_2, "Tell me about the features of  GammaAir")
execute_prompt("USER 1", config_1, "What is its price ?")
execute_prompt("USER 2", config_2, "What is its price ?")




USER 1: We currently carry the SpectraBook S. Here are its features:

- Processor: Intel Core i9 (workstation-class performance)
- Memory: 64GB RAM for heavy multitasking
- Storage: 2TB SSD for fast load times and large project files
- Use case: Designed for power users; great for video editing, 3D rendering, and other intensive workloads

Did you mean the SpectraBook S? If so, I can also share pricing or compare it with other models.

USER 2: Here’s what we have on the GammaAir series:

Model: GammaAir X
- Processor: AMD Ryzen 7
- Memory: 32GB DDR4
- Storage: 512GB NVMe SSD
- Design: Thin-and-light chassis
- Key advantage: High performance in a highly portable design

Did you have a specific GammaAir variant in mind, or would you like more details on the GammaAir X?

USER 1: The SpectraBook S is priced at $2,499. Would you like to check availability or compare it with similar models?

USER 2: GammaAir X is priced at $1,399. Would you like to add it to your cart or compare it with ano